In [1]:
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler 
from telegram import InlineKeyboardMarkup, InlineKeyboardButton 
import random
# ===============================================================================================
with open('token.txt') as file:
    TOKEN= file.read()
    file.close()
# ===============================================================================================
updater = Updater(token=TOKEN, use_context=True)
dispatcher = updater.dispatcher
# ===============================================================================================
# 你們老師的講義有寫
# update.message.reply_text 可以直接回傳你第一個放的字串
# 建議以後要讓機器人回話要用 update.message.reply_text(你要他回的字串)
#
# reply_markup = InlineKeyboardMarkup([[ InlineKeyboardButton("洗牌",callback_data = 你要回傳的資料 )]]))
# 你們老師的講義也有寫
# ===============================================================================================
# 參數區
main = []
others = []
stock = []
# ===============================================================================================
# 控制區
def start(update, context):
    text = newGame()
    update.message.reply_text(text,
            reply_markup = InlineKeyboardMarkup([
                [InlineKeyboardButton("洗牌", callback_data = "洗牌"),
                InlineKeyboardButton("重玩", callback_data = "重玩")]
            ]))

def move(update, context):
    response = moving(update["message"]["text"])
    text = toString() + "\n" + response
    update.message.reply_text(text,
            reply_markup = InlineKeyboardMarkup([
                [InlineKeyboardButton("洗牌", callback_data = "洗牌"),
                InlineKeyboardButton("重玩", callback_data = "重玩")]
            ]))

def button(update, context):    
    update.callback_query.edit_message_text(judgement(update.callback_query.data),
            reply_markup = InlineKeyboardMarkup([
                [InlineKeyboardButton("洗牌", callback_data = "洗牌"),
                InlineKeyboardButton("重玩", callback_data = "重玩")]
            ]))    
    
def judgement(callback_data):
    if callback_data == "洗牌":
        return openOthers()
    else :
        return newGame()

    
button_handler = CallbackQueryHandler(button)
dispatcher.add_handler(button_handler)      

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)


move_handler = CommandHandler('move', move)
dispatcher.add_handler(move_handler)

# ===============================================================================================
# 操作區
def openOthers():
    if len(others) != 0:
        temp = others[0]
        others.remove(temp)
        others.append(temp)
        return toString()
    else:
        return "沒有牌啦 哭阿"
def moving(row):
    _,get,index,to = row.split(" ")
    get = int(get)
    index = int(index)
    to = int(to)
    ###### index 的判定
    #首先先判定 get 是否為 4 代表他從雜牌區來的
    if get == 4:
        response = moveFromOthers(get,index,to)
    ##### 再來是決定下面牌區 row 跟 row 的交換
    ### 這裡啊 其實寫成 try exception 會比較好 但是算了 就一個一個偵錯吧
    ### index 長度小於 0 以及 該翻開的牌為 覆蓋的牌
            ####下面是判定 index 有沒有小於 0  ####下面是判定 index 有沒有大於列的長度      ####下面是判定 index 是否為覆蓋的牌 
    elif len(main[get-5]) - index >= 0 or len(main[get-5]) - index < len(main[get-5]) or main[get-5][len(main[get-5]) - index] == "🂻":
        if to > 4 and get > 4:
            response = moveBetweenRows(get,index,to)
        ##### 最後就是代表是要從 下面牌區 移到上面牌區的
        elif to < 4 and get > 4 :
            ##### 從 get 移動的牌 必須要是從右邊數過來第一張
            if index == 1 :
                response = moveToStock(get,index,to)
            else:
                response = "你輸入的移動方式有問題 別搞我"
    ##### 代表他填的有問題
    else:
        response = "你輸入的移動方式有問題 別搞我"
    return response
def moveBetweenRows(get,index,to):
    #調整index 只要輸入第幾張牌即可 不必從 0 開始
    index = len(main[get-5]) - index
    obj = main[get-5][index:]
    flag = True
    #先判斷能不能移：
    # 1.如果是空列的情形 直接貼上。
    #看要不要加入 [若為空的列則只能放 13] 的規則。
    if len(main[to-5]) == 0 :
        main[to-5] += obj
    # 2.非空的話 判定能不能接上去。
    # 對 to 的最後一張 跟 obj 的第 0 張 進行判斷。
    # 我使用[1:]取得後面兩位也就是數字位 並轉成 int 進行比對。
    elif int(main[to-5][-1]["card"][1:]) - 1 == int(obj[0]["card"][1:]):
        main[to-5] += obj
    else:
        response = "不可以這樣移 是在哭阿"
        flag = False
    if flag:   
        #移動完後才判斷是否能翻開牌 1.不可小於 0 , 2.為覆蓋的牌
        if index - 1 >= 0 and not main[get-5][index-1]["status"]:
            main[get-5][index-1]["status"] = True
        #最後才將 get 的值進行更改
        main[get-5] = main[get-5][:index]
        response = "移的好"
    return response
    
def moveToStock(get,index,to):
    #調整index 只要輸入第幾張牌即可 不必從 0 開始
    index = len(main[get-5]) - index
    # [-1]代表的是 main[get-5]陣列的最後一張 也等於選取畫面的最右邊那一張
    obj = main[get-5][-1]
    flag = True
    #注意 stock 我改成 dictionary 類型了 !注意!這裡有眉角
    #眉角：
    # dictionary 的keys()方法不能直接用[]去取值 因為他不是 list 所以我先把他變成 list，
    # 用成 list 之後就可以直接用 to 去取值了。
    # 先判斷能不能移：1. stock 是否為空的 2. obj 是否為 1。
    # 我使用[1:]取得後面兩位也就是數字位 並轉成 int 進行比對，。
    if len(stock[list(stock.keys())[to]]) == 0:
        #之後進行花色的對比
        if obj["card"][:1] == list(stock.keys())[to] and int(obj["card"][1:]) == 1 :
            stock[list(stock.keys())[to]].append(obj["card"])
        else:
            response = "不可以這樣移 是在哭阿"
            flag = False
    #再來是非空情況
    #前面是比較花色有沒有對到 後面是比較數字有沒有相接 
    elif obj["card"][:1] == list(stock.keys())[to] and int(stock[list(stock.keys())[to]][-1][1:]) + 1 == int(obj["card"][1:]):
        stock[list(stock.keys())[to]].append(obj["card"])
    else:
        # 如果都不符合的話對 flag 進行更改
        response = "不可以這樣移 是在哭阿"
        flag = False
    if flag:
        #確認可以移動後才開始翻牌
        #判斷是否能翻開牌 1.不可小於 0 , 2.為覆蓋的牌
        if index - 1 >= 0 and not main[get-5][index-1]["status"]:
            main[get-5][index-1]["status"] = True
        #最後才將 get 的值進行更改
        main[get-5] = main[get-5][:index]
        response = "移的妙"
    return response

def moveFromOthers(get,index,to):
    obj = others[-1]
    flag = True
    #基本上跟上面兩個 function 一樣 個別判斷移去哪裡
    #移到下面牌區
    if to > 4 :
        #判斷能不能移 基本上跟上面一樣
        #先判斷能不能移：
        # 1.如果是空列的情形 直接貼上。
        #看要不要加入 [若為空的列則只能放 13] 的規則。
        if len(main[to-5]) == 0 :
            main[to-5].append({"status":True,"card":obj})
        # 2.非空的話 判定能不能接上去。
        # 對 to 的最後一張 跟 obj 的第 0 張 進行判斷。
        # 我使用[1:]取得後面兩位也就是數字位 並轉成 int 進行比對。
        ###此處因為 others 直接提供卡牌內容 所以直接使用[1:]取得後面兩位
        elif int(main[to-5][-1]["card"][1:]) - 1 == int(obj[1:]):
            main[to-5].append({"status":True,"card":obj})
        else:
            response = "不可以這樣移 是在哭阿"
            flag = False
        if flag:
            #移動後移掉 obj
            others.remove(obj)
            response = "移的呱呱叫"
    #移到上面四花色區
    else:
        # 先判斷能不能移：1. stock 是否為空的。
        # 我使用[1:]取得後面兩位也就是數字位 並轉成 int 進行比對。
        ###此處因為 others 直接提供卡牌內容 所以直接使用[1:]取得後面兩位
        if len(stock[list(stock.keys())[to]]) == 0:
            # 跟上面一樣是看花色有沒有對到 數字是否為一
            if obj[:1] == list(stock.keys())[to] and int(obj[1:]) == 1:
                stock[list(stock.keys())[to]].append(obj)
            else:
                response = "不可以這樣移 是在哭阿"
                flag = False
        #再來是非空情況
        ###此處因為 others 直接提供卡牌內容 所以直接使用[1:]取得後面兩位
        # 跟上面一樣判定 1. 是否相同花色 2. 是否為連續的牌
        elif obj[:1] == list(stock.keys())[to] and int(stock[list(stock.keys())[to]][-1][1:]) + 1 == int(obj[1:]) :
            stock[list(stock.keys())[to]].append(obj["card"])
        else:
            response = "不可以這樣移 是在哭阿"
            flag = False
        if flag:
            #移動後移掉 obj
            others.remove(obj)
            response = "移的呱呱叫"
    return response
# ===============================================================================================
# 開始新的一局
def newGame():
    global main
    global stock
    global others
    # 產生牌組
    deck = [house + str(num).zfill(2) for house in ['♥','♣','♦','♠'] for num in range(1,14)]
    # 發牌
    chosen = random.sample(deck,28)
    # 給予翻牌區
    for x in chosen:
        deck.remove(x)
    others = deck
    # 給予主要牌區
    localMain = [[],[],[],[],[],[],[]]
    flag = 0
    for x in range(1,8):
        for y in range(x):
            localMain[x-1].append({"status" : y == x - 1 ,"card" : chosen[flag]})
            flag += 1
    main = localMain
    # 四種花色
    stock = {'♠':[],'♥':[],'♦':[],'♣':[]}
    return toString()
# ===============================================================================================
# 轉換成輸出文字
def toString():
    # 統一由 flag 0 開始
    flag = 0
    front = ""
    # 前面花排
    for x in stock.keys():
        front += "{0} row({1:02d}):".format(x,flag)
        if len(stock[x]) != 0 :
            front += stock[x][-1]
        front += "\n"
        flag += 1
    # 雜排
    front += "\n🂠 row({:02d}):".format(flag)
    if len(others) != 0 :
        front += others[-1]
    front += "\n\n"
    flag += 1
    # 後面的排
    for x in main:
        front += "🂻 row({:02d}):".format(flag)
        for y in x:
            if y["status"]:
                front += y["card"]
            else:
                front += "🂻"
        front += "\n"
        flag += 1
    return front    

# ===============================================================================================
updater.start_polling()

In [104]:
updater.stop()